참고 링크: https://github.com/TeamOTK/Search/blob/main/src/Search.py

In [1]:
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

PDF에서 텍스트를 읽어와서 청크로 분할

In [5]:
reader = PdfReader("data/spiderman1.pdf")
chunks = []
chunk_length = 1000

for page in reader.pages:
    text_page = page.extract_text()
    chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

In [7]:
reader = PdfReader("data/spiderman2.pdf")

for page in reader.pages:
    text_page = page.extract_text()
    chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

In [8]:
len(chunks)

94

크로마디비에 벡터 저장

In [10]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
# embeddings = OpenAIEmbeddings()
# vectorstore = FAISS.from_texts(chunks, embeddings)

In [13]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_texts(chunks, embeddings)

In [4]:
retriever = vectorstore.as_retriever()

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True,
    output_key="output",
)

GPT4 : gpt-4-1106-preview
파인튜닝 1: ft:gpt-4o-mini-2024-07-18:personal:spiderman-1:9s1amhcg

In [6]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="ft:gpt-4o-mini-2024-07-18:personal:spiderman-1:9s1amhcg",
    temperature=0
)

In [7]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    verbose=True,
    return_source_documents=True
)

In [8]:
from langchain.agents import Tool

tools = [
    Tool(
        name="doc_search_tool",
        func=qa,
        description=(
            "This tool is used to retrieve information from the knowledge base"
        )
    )
]

In [9]:
system_message = """
    You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        -You must answer in Koreans
        -You must answer like you're spider-man
        -Use a first-person perspective. Do not say "peter parker ~"
"""

In [10]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=memory,
    return_source_documents=True,
    return_intermediated_steps=True,
    agent_kwargs={"system_message": system_message}
)

c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
query = "피터, 다른 우주의 피터 파커를 만났을 때 무슨 일을 했어?"
result = agent(query)
print(result["output"])

c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
피터 파커는 다른 우주의 피터 파커와 만나서 서로의 경험을 공유하고, 함께 적들과 싸우며 협력했어.


참고
https://rimiyeyo.tistory.com/entry/RAG%EB%A5%BC-%EC%88%98%ED%96%89%ED%95%98%EA%B8%B0-%EC%9C%84%ED%95%9C-LangChain%EC%9D%98-RetrievalQA-%EA%B5%AC%EC%A1%B0%EC%99%80-%EA%B5%AC%ED%98%84%EB%B0%A9%EB%B2%95